In [2]:
import scipy.optimize as opt
import scipy.linalg as sla
import math
import numpy as np
import pprint
import random

from IPython.display import display, Math, Latex

In [3]:
inverse = lambda A: sla.solve(A, np.eye(A.shape[0]))

In [4]:
def numpy_to_pmatrix(a):
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{pmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{pmatrix}']
    return '\n'.join(rv)

In [5]:
def is_float_vector(x):
    for idx, x_i in enumerate(x):
        if not math.isclose(x_i, math.floor(x_i)):
            return x_i, idx
    else:
        return None, None

In [6]:
def is_float_vector_2(x):
    float_values = []
    for idx, x_i in enumerate(x):
        if not math.isclose(x_i, math.floor(x_i)):
            float_values.append((x_i, idx))
    if float_values:
        a = random.choice(float_values)
        print(a)
        return a
    else:
        return None, None

In [7]:
def get_basis_and_non_basis(x):
    basis = []
    non_basis = []
    for idx, x_i in enumerate(x):
        basis.append(idx) if x_i else non_basis.append(idx)
    return np.array(basis), np.array(non_basis)

In [8]:
def get_new_condition(l, basis, size):
    new_line = []
    idx = 0
    for i in range(size):
        if i in basis:
            new_line.append(l[idx])
            idx += 1
        else:
            new_line.append(0)
    else:
        new_line.append(-1)
    return np.array(new_line)

In [9]:
def frac(x):
    print(x)
    return np.array([x_i - math.floor(x_i) for x_i in x.ravel()])

In [10]:
def solve(A, c, b):
    need_size = c.size
    while True:
        resolved = opt.linprog(c=c, A_eq=A, b_eq=b, method='simplex')
        
        if resolved.success == False:
            display(Math(r'\text{Данная задача ILP: }\\A = '  + numpy_to_pmatrix(A)))
            display(Math(r'b = '  + numpy_to_pmatrix(b)))
            display(Math(r'c = '  + numpy_to_pmatrix(c)))
            display(Math(r'\\\text{ не имеет решения потому что: }\\'))
            print(resolved.message)
            return None, None
        print("x")
        print(resolved.x)
        print(-resolved.fun)
        number_to_round, idx = is_float_vector_2(resolved.x)
        print(f"number_to_round {number_to_round}")
        not_solution = bool(number_to_round)
        if not not_solution:
            break
        basis, non_basis = get_basis_and_non_basis(resolved.x)
        k, = np.where(basis == idx)
        A_basis = A[:, basis]
        print(A_basis)
        inverse_A_basis = inverse(A_basis)
        A_non_basis = A[:, non_basis]
        
        L = inverse_A_basis @ A_non_basis
        print("L")
        print(L)
        l = frac(L[k])
        l = np.append(l, -1)
        print("l")
        print(l)
        print(A)
        A = np.append(A, np.zeros(b.size).reshape(-1, 1), axis=1)
        print(A)
        l = get_new_condition(l, non_basis, c.size)
        print(l)
        A = np.append(A, [l], axis=0)
        
        b_i = number_to_round - math.floor(number_to_round)
        b = np.append(b, b_i)
        
        c = np.append(c, 0)
        
        print(A)
        print(b)
        print(c)
        return resolved.x[:need_size], -resolved.fun
    return resolved.x[:need_size], -resolved.fun

In [11]:
A = np.array([
    [-4, 6, 1, 0],
    [1, 1, 0, 1]
], 
    dtype="float32"
)
b = np.array([9, 4], dtype="float32")
c = np.array([-1, 2, 0, 0], dtype="float32")

In [12]:
solve(A, c, b)

x
[ 4.  0. 25.  0.]
4.000000000000001
number_to_round None


(array([ 4.,  0., 25.,  0.]), 4.000000000000001)

In [13]:
A = np.array([
    [3, 2, 1, 0],
    [-3, 2, 0, 1]
], 
    dtype="float32"
)
b = np.array([6, 0], dtype="float32")
c = np.array([0, 1, 0, 0], dtype="float32")

In [14]:
solve(A, -c, b)

x
[1.  1.5 0.  0. ]
1.5
(1.5, 1)
number_to_round 1.5
[[ 3.  2.]
 [-3.  2.]]
L
[[ 0.16666667 -0.16666667]
 [ 0.25        0.25      ]]
[[0.25 0.25]]
l
[ 0.25  0.25 -1.  ]
[[ 3.  2.  1.  0.]
 [-3.  2.  0.  1.]]
[[ 3.  2.  1.  0.  0.]
 [-3.  2.  0.  1.  0.]]
[ 0.    0.    0.25  0.25 -1.  ]
[[ 3.    2.    1.    0.    0.  ]
 [-3.    2.    0.    1.    0.  ]
 [ 0.    0.    0.25  0.25 -1.  ]]
[6.  0.  0.5]
[-0. -1. -0. -0.  0.]


(array([1. , 1.5, 0. , 0. ]), 1.5)

In [15]:
A_prime = np.concatenate((A, np.eye(A.shape[0])), axis=1)
c_prime = np.concatenate((c, np.zeros(A.shape[0])), axis=0)
initial_solution = np.array([0] * c.size + b.tolist())
initial_basis = set(list(range(A_prime.shape[1]))) - set(list(range(A.shape[1])))
print(initial_basis)

{4, 5}


In [16]:
list(range(9))

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [17]:
initial_basis = np.argwhere(initial_solution != 0).ravel()

In [18]:
initial_basis

array([4])

In [19]:
np.argwhere(initial_solution != 0)

array([[4]])

In [20]:
c = np.array([-1, 2, 0, 0], dtype="float32")